In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt

mouse_list = ["ID138464-1","ID138464-2","ID138464-3","ID138464-4","ID138464-5","ID138464-6"]

phase_list=["A1","A2","A31","A32","A41","A42","A5","A6","A7"]

mouse = mouse_list[5]

for ph in range(0,9):
    
    phase=phase_list[ph]

    #for index in range(0,len(Index)):

    inputXY='ID138464-%s_%s_XY.txt' %(mouse,phase)

    dfx = pd.read_table(inputXY,header=0)

    inputT='ID138464-%s_long_trace_%s.csv'%(mouse,phase)

    dfl=pd.read_csv(inputT,header=0)

    dfx["session"]=0

    ses=dfx.columns.get_loc('session')

    for i in range(1,dfx.shape[0]):

        if (dfx.iloc[i].loc["FZ"]!=dfx.iloc[i-1].loc["FZ"]):

            dfx.iloc[i,ses]= int(dfx.iloc[i-1,ses])+1

        else:

            dfx.iloc[i,ses]= int(dfx.iloc[i-1,ses])

    dfx['FZ_c0']=dfx['FZ']
    dfx['ses_c0']=dfx['session']
    ses_cnt=0
    FZ_col=dfx.columns.get_loc('FZ_c0')
    session_col=dfx.columns.get_loc('ses_c0')
    ses_c='ses_c%s'%(ses_cnt)

    mconseq='mconseq_%s'%(ses_cnt)
    dfx[mconseq]=0
    mconseq_col=dfx.columns.get_loc('mconseq_0')
    Slice_col=dfx.columns.get_loc('Slice')

    for i in range (0,dfx.shape[0]):

        ses_no = dfx.iloc[i,session_col]

        if dfx.iloc[i,FZ_col] == 0:

            if ses_no == dfx.iloc[dfx.shape[0]-1].loc[ses_c]:

                dfx.iloc[i,mconseq_col]=-100

                break;

            df_ns=dfx[dfx[ses_c]==ses_no+1]

        else:

            df_ns=dfx[dfx[ses_c]==ses_no]

        df_ns1=df_ns.iloc[0]

        dfx.iloc[i,mconseq_col]=int(dfx.iloc[i,Slice_col]-df_ns1.iloc[Slice_col]+1)

    valid_fz='valid_%s'%(ses_cnt)

    dfx[valid_fz]=0

    valid_col=dfx.columns.get_loc(valid_fz)

    for i in range(0,dfx.shape[0]):

        ses_no_v = dfx.iloc[i,session_col]

        if dfx.iloc[i,FZ_col] == 0:

            if ses_no_v == dfx.iloc[dfx.shape[0]-1].loc[ses_c]:

                break;

            df_ns_v=dfx[dfx[ses_c]==ses_no_v+1]

        else:

            df_ns_v=dfx[dfx[ses_c]==ses_no_v]

        df_ns1_v=df_ns_v.iloc[df_ns_v.shape[0]-1]


        if df_ns1_v[mconseq_col]<9:

            dfx.iloc[i,valid_col]=0

        else:

            dfx.iloc[i,valid_col]=1

    for ses_cnt in range(1,10):
        FZ_c='FZ_c%s'%(ses_cnt)
        FZ_cm='FZ_c%s'%(ses_cnt-1)
        ses_c='ses_c%s'%(ses_cnt)
        ses_cm='ses_c%s'%(ses_cnt-1)
        dfx[FZ_c]=dfx[FZ_cm]
        dfx[ses_c]=0

        fzc=dfx.columns.get_loc(FZ_c)

        cnt=0

        sup=0

        r=20

        for i in range(1,int(dfx.iloc[dfx.shape[0]-1].loc[ses_cm])+1):

            fz=dfx[dfx[ses_cm]==i].iloc[0,fzc]

            nfz=int((1-fz)*(1-fz))

            slice_b=dfx[dfx[ses_cm]==i].iloc[0].loc['Slice']

            slice_a=dfx[dfx[ses_cm]==i].iloc[dfx[dfx[ses_cm]==i].shape[0]-1].loc['Slice']

            Queryp='%d<=Slice<%d'%(int(slice_b),int(slice_a+r))

            Querym='%d<=Slice<%d'%(int(slice_b),int(slice_a+r))

            dfx_sp=dfx.query(Queryp)

            dfx_sm=dfx.query(Querym)

            if(i<int(dfx.iloc[dfx.shape[0]-1].loc[ses_cm])):

                pp=1.0*dfx_sp[FZ_cm].value_counts()[fz]/(dfx_sp.shape[0]*1.0)

                pm=1.0*dfx_sm[FZ_cm].value_counts()[fz]/(dfx_sm.shape[0]*1.0)

            else:

                p=1

            if((dfx[dfx[ses_cm]==i].shape[0]<10) and (pp<0.350 or pm<0.350) and  (fz==0)):

                dfx.loc[dfx[ses_cm]==i,ses_c]=cnt

                dfx.loc[dfx[ses_cm]==i,FZ_c]=nfz

                sup=1

            else:

                cnt+=1-sup

                dfx.loc[dfx[ses_cm]==i,ses_c]=cnt

                dfx.loc[dfx[ses_cm]==i,FZ_c]=fz

                sup=0

        mconseq='mconseq_%s'%(ses_cnt)
        dfx[mconseq]=-10

        FZ_col=dfx.columns.get_loc(FZ_c)
        session_col=dfx.columns.get_loc(ses_c)
        mconseq_col=dfx.columns.get_loc(mconseq)
        Slice_col=dfx.columns.get_loc('Slice')

        for i in range (0,dfx.shape[0]):

            ses_no = dfx.iloc[i,session_col]

            if dfx.iloc[i,FZ_col] == 0:

                if ses_no == dfx.iloc[dfx.shape[0]-1].loc[ses_c]:

                    break;

                df_ns=dfx[dfx[ses_c]==ses_no+1]

            else:

                df_ns=dfx[dfx[ses_c]==ses_no]

            df_ns1=df_ns.iloc[0]

            dfx.iloc[i,mconseq_col]=int(dfx.iloc[i,Slice_col]-df_ns1.iloc[Slice_col]+1)

        valid_fz='valid_%s'%(ses_cnt)

        dfx[valid_fz]=0

        valid_col=dfx.columns.get_loc(valid_fz)

        for i in range(0,dfx.shape[0]):

            ses_no_v = dfx.iloc[i,session_col]

            if dfx.iloc[i,FZ_col] == 0:

                if ses_no_v == dfx.iloc[dfx.shape[0]-1].loc[ses_c]:

                    break;

                df_ns_v=dfx[dfx[ses_c]==ses_no_v+1]

            else:

                df_ns_v=dfx[dfx[ses_c]==ses_no_v]

            df_ns1_v=df_ns_v.iloc[df_ns_v.shape[0]-1]


            if df_ns1_v[mconseq_col]<9:

                dfx.iloc[i,valid_col]=0

            else:

                dfx.iloc[i,valid_col]=1

        if(dfx.iloc[dfx.shape[0]-1].loc[ses_cm]==dfx.iloc[dfx.shape[0]-1].loc[ses_c]):

            break;

    dfx_m=pd.DataFrame(columns=dfx.columns,index=np.arange(0,int(dfx.iloc[dfx.shape[0]-1].loc['Slice']-1)*10))

    for i in range(2,int(dfx.shape[0])):

        for j in range(0,10):

            for k in range(0,len(dfx_m.columns)):

                dfx_m.iloc[(i-2)*10+j].loc[dfx_m.columns[k]]=dfx.iloc[i].loc[dfx_m.columns[k]]

    df=pd.concat([dfx_m, dfl], axis=1)
    df=df.dropna(how='any')

    Index=np.arange(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1),1)

    df_mean=pd.DataFrame(index=Index,columns=[])

    for i in range(0,len(df.columns)):

        df_mean[df.columns[i]]=-100

    for i_n in range(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1)):

        i = i_n - int(df.iloc[0].loc['Slice'])

        df_mean.iloc[i].loc['Slice']=i_n

        dfs = df[df['Slice']==i_n]

        for k in range(0,df.columns.get_loc('Time2')+1):

            df_mean.iloc[i,k]=dfs.iloc[0,k]


        for j in range(dfs.columns.get_loc(' C000'),dfs.columns.get_loc(' C261')+1):

            df_mean.iloc[i,j]=dfs[dfs.columns[j]].mean()

        for l in range(dfs.columns.get_loc('session'),dfs.columns.get_loc(ses_c)+1):

            df_mean.iloc[i,l]=int(dfs.iloc[0,l])

    dfmean=df_mean

    Index=np.arange(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1),1)

    df_max=pd.DataFrame(index=Index,columns=[])

    for i in range(0,len(df.columns)):

        df_max[df.columns[i]]=-100

    for i_n in range(int(df.iloc[0].loc['Slice']),int(df.iloc[df.shape[0]-1].loc['Slice']+1)):

        i = i_n - int(df.iloc[0].loc['Slice'])

        df_max.iloc[i].loc['Slice']=i_n

        dfs = df[df['Slice']==i_n]

        for k in range(0,df.columns.get_loc('Time2')+1):

            df_max.iloc[i,k]=dfs.iloc[0,k]


        for j in range(dfs.columns.get_loc(' C000'),dfs.columns.get_loc(' C261')+1):

            df_max.iloc[i,j]=dfs[dfs.columns[j]].max()

        for l in range(dfs.columns.get_loc('session'),dfs.columns.get_loc(ses_c)+1):

            df_max.iloc[i,l]=int(dfs.iloc[0,l])

    dfmax=df_max

    for j in range(dfmean.columns.get_loc(' C000'),dfmean.columns.get_loc(' C261')+1):

        plt.plot(dfmean['Time1'],dfmean[dfmean.columns[j]],color='red')

        plt.plot(dfmax['Time1'],dfmax[dfmax.columns[j]],color='black')

        plt.ylim=(-0.05,0.30)

        for i in range(0,dfmean.shape[0]):

            if dfmean.iloc[i].loc['FZ'] == 1:

                if dfmean.iloc[i].loc['X']  <= 125:

                    if dfmean.iloc[i].loc['Y']  <=100:

                        Color='lightpink'

                    else:

                        Color='lightblue'

                else:

                    if dfmean.iloc[i].loc['Y']  <=100:

                        Color='silver'

                    else:

                        Color='lightgreen'


                plt.fill([dfmean.iloc[i].loc['Time1'],dfmean.iloc[i].loc['Time1']+1,dfmean.iloc[i].loc['Time1']+1,dfmean.iloc[i].loc['Time1']],[-0.05,-0.05,0.30,0.30],color=Color)

            if (dfmean.iloc[i].loc['Shock'] == 1):

                plt.plot([dfmean.iloc[i].loc['Time1'],dfmean.iloc[i].loc['Time1']],[-0.05,0.30],color='red',linestyle='dashed')

        plt.xlabel('Time1')
        plt.ylabel('Ca2+Img')             
        title_name='fig1/sliceini_phase%s_mouse%s_%s' % (phase,mouse,dfmean.columns[j])
        plt.title(title_name)
        file_name=title_name+'.png'
        plt.savefig(file_name)
        plt.figure(figsize=(5.4, 3.6), dpi=80)
        #plt.show()

    for j in range(dfmean.columns.get_loc(' C000'),dfmean.columns.get_loc(' C261')+1):

        plt.plot(dfmean['Time1'],dfmean[dfmean.columns[j]],color='red')

        plt.plot(dfmax['Time1'],dfmax[dfmax.columns[j]],color='black')

        plt.ylim=(-0.05,0.30)

        for i in range(0,dfmean.shape[0]):

            if dfmean.iloc[i].loc[FZ_c] == 1:

                if dfmean.iloc[i].loc['X']  <= 125:

                    if dfmean.iloc[i].loc['Y']  <=100:

                        Color='lightpink'

                    else:

                        Color='lightblue'

                else:

                    if dfmean.iloc[i].loc['Y']  <=100:

                        Color='silver'

                    else:

                        Color='lightgreen'


                plt.fill([dfmean.iloc[i].loc['Time1'],dfmean.iloc[i].loc['Time1']+1,dfmean.iloc[i].loc['Time1']+1,dfmean.iloc[i].loc['Time1']],[-0.05,-0.05,0.30,0.30],color=Color)

            if (dfmean.iloc[i].loc['Shock'] == 1):

                plt.plot([dfmean.iloc[i].loc['Slice'],dfmean.iloc[i].loc['Slice']],[-0.05,0.30],color='red',linestyle='dashed')


        plt.xlabel('Time1')
        plt.ylabel('Ca2+Img')    
        title_name='fig1/sliceope_phase%s_mouse%s_%s' % (phase,mouse,dfmean.columns[j])
        plt.title(title_name)
        file_name=title_name+'.png'
        plt.savefig(file_name)
        plt.figure(figsize=(5.4, 3.6), dpi=80)
        #plt.show()

    for j in range(df.columns.get_loc(' C000'),df.columns.get_loc(' C261')+1):   

        df_m=df[(df[mconseq]<0)]

        df_p=df[(df[mconseq]>=0) & (df[valid_fz]==1)]

        plt.hist([df_m[df.columns[j]],df[df.columns[j]],df_p[df.columns[j]]],bins=10,normed=True,label=['nonFZ','ALL','FZ'])
        #plt.hist(df_m[df.columns[j]])
        plt.xlabel('Ca2+img')
        plt.ylabel('density')

        title_name='fig1/histope_phase%s_mouse%s_%s' % (phase,mouse,dfmean.columns[j])
        plt.title(title_name)
        file_name=title_name+'.png'
        plt.savefig(file_name)
        plt.figure(figsize=(5.4, 3.6), dpi=80)
        #splt.show()

    for j in range(df.columns.get_loc(' C000'),df.columns.get_loc(' C261')+1):   

        df_m=df[(df['mconseq_0']<0)]

        df_p=df[(df['mconseq_0']>=0) & (df['valid_0']==1)]

        plt.hist([df_m[df.columns[j]],df[df.columns[j]],df_p[df.columns[j]]],bins=10,normed=True,label=['nonFZ','ALL','FZ'])
        #plt.hist(df_m[df.columns[j]])
        plt.xlabel('Ca2+img')
        plt.ylabel('density')

        title_name='fig1/histini_phase%s_mouse%s_%s' % (phase,mouse,dfmean.columns[j])
        plt.title(title_name)
        file_name=title_name+'.png'
        plt.savefig(file_name)
        plt.figure(figsize=(5.4, 3.6), dpi=80)
        #splt.show()

/Users/kanri/anaconda/envs/jupyter-env/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/kanri/anaconda/envs/jupyter-env/lib/python3.6/site-packages/ipykernel_launcher.py:248: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kanri/anaconda/envs/jupyter-env/lib/python3.6/site-packages/ipykernel_launcher.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/kanri/anaconda/envs/ju